In [1]:
from pyspark.sql import SparkSession
from pyspark import SQLContext
bucket = "onedayproject"  

# hive table 에 데이터를 넣기 위한 옵션 동적파티션
spark = SparkSession.builder.appName("TLC_analysis_glue_hive").enableHiveSupport()\
                .config("hive.exec.dynamic.partition", "true")\
                .config("hive.exec.dynamic.partition.mode", "nonstrict")\
                .getOrCreate()
                
# %%configure -f
# {"conf":{"spark.dynamicAllocation.enabled":"false", "conf":"ww",~~~~}}


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1644337534585_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
trips_df = spark.read.load("s3://"+ bucket +"/tlc_tripsdata/yellow_tripdata_2021-01.csv",
                     format="csv", sep=",", inferSchema="true", header="true")
zone_df =  spark.read.load("s3://"+ bucket +"/tlc_zonedata/taxi+_zone_lookup.csv",
                     format="csv", sep=",", inferSchema="true", header="true")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
trips_df.createOrReplaceTempView("trips_view")
zone_df.createOrReplaceTempView("zone_view")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
trips_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)

In [13]:
spark.sql("select * from trips_view limit 10").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2021-01-01 00:30:10|  2021-01-01 00:36:12|              1|          2.1|         1|                 N|         142|          43|           2|        8.0|  3.0|    0.5|       0.0|         0.0|                  0.3

In [14]:
spark.sql("select * from zone_view limit 10").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
+----------+-------------+--------------------+------------+

In [4]:
trips_silver_df = sqlContext.sql("""
SELECT 
    passenger_count,
    PULocationID as pickup_location_id,
    DOLocationID as dropoff_location_id,
    trip_distance,
    HOUR(tpep_pickup_datetime) as pickup_time,
    DATE_FORMAT(TO_DATE(tpep_pickup_datetime), 'EEEE') AS day_of_week,
    total_amount
FROM
    trips_view
WHERE
    total_amount < 5000
    AND total_amount > 0
    AND trip_distance > 0
    AND trip_distance < 500
    AND passenger_count < 4
    AND TO_DATE(tpep_pickup_datetime) >= '2021-01-01'
    AND TO_DATE(tpep_pickup_datetime) < '2021-01-31'
""")

trips_silver_df.createOrReplaceTempView("trips_silver_df")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
trips_silver_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- passenger_count: integer (nullable = true)
 |-- pickup_location_id: integer (nullable = true)
 |-- dropoff_location_id: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)

In [12]:
sqlContext.cacheTable("trips_silver_df")
sqlContext.cacheTable("zone_view")
# 캐싱을 한테이블과 안한 테이블이 있으면 안한테이블 기준으로 로드하기때문에
# 반드시 테이블은 캐싱해주고 밑 sql을 해야 모든 데이터가 메모리에 올라감
# 올라가면 빠르게 분석가능

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
%%sql
SELECT count(*) from trips_silver_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [ ]:
# hiva table 생성 (glue)

In [13]:
%%sql
CREATE EXTERNAL TABLE tlc_glue_database.trips_silver_view
(
    passenger_count int,
    pickup_location_id int,
    dropoff_location_id int,
    trip_distance double,
    pickup_time int,
    total_amount double
)
PARTITIONED BY (
    day_of_week string
)
STORED AS PARQUET
LOCATION "s3://onedayproject/hive/silver/trips_silver_view"
tblproperties ("parquet.compress"="SNAPPY" ,"classification"="parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
"Table or view 'trips_silver_view' already exists in database 'tlc_glue_database';"
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/session.py", line 767, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery), self._wrapped)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 71, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: "Table or view 'trips_silver_view' already exists in database 'tlc_glue_database';"



In [16]:
# hive table 에 insert 이렇게 저장하는 방식이 바로 파케이로 저장하는것보다
# 10배미만으로 빠름
sqlContext.sql("""
    INSERT overwrite table tlc_glue_database.trips_silver_view
    PARTITION(day_of_week)
    SELECT 
            passenger_count,
            pickup_location_id,
            dropoff_location_id,
            trip_distance,
            pickup_time,
            total_amount,
            day_of_week
    FROM
            trips_silver_df
""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [17]:
%%sql
select * from tlc_glue_database.trips_silver_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [20]:
# glod data (glue에 저장된 silver data와 code table 과의 join)
trips_glod_df = sqlContext.sql("""
SELECT 
    s.passenger_count,
    s.pickup_location_id,
    s.dropoff_location_id,
    s.trip_distance,
    s.pickup_time,
    s.total_amount,
    pz.Zone as pickup_zone,
    dz.Zone as dropoff_zone
FROM 
    tlc_glue_database.trips_silver_view s
    LEFT JOIN 
        zone_view pz
    ON
        s.pickup_location_id = pz.LocationID
    LEFT JOIN
        zone_view dz
    ON 
        s.dropoff_location_id = dz.LocationID
""")

trips_glod_df.createOrReplaceTempView("trips_gold_df")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
spark.sql("select * from trips_gold_df limit 10").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+------------------+-------------------+-------------+-----------+------------+--------------------+--------------------+
|passenger_count|pickup_location_id|dropoff_location_id|trip_distance|pickup_time|total_amount|         pickup_zone|        dropoff_zone|
+---------------+------------------+-------------------+-------------+-----------+------------+--------------------+--------------------+
|              0|               140|                164|          2.6|          0|        13.8|     Lenox Hill East|       Midtown South|
|              1|               132|                130|          6.1|          0|        20.3|         JFK Airport|             Jamaica|
|              1|               132|                256|         16.1|          0|       55.35|         JFK Airport|Williamsburg (Sou...|
|              1|               164|                164|          0.7|          0|         9.3|       Midtown South|       Midtown South|
|              1|               26

In [22]:
jdbc_hostname = ""
jdbc_database = "dm"
username=""
password=""
jdbc_port = 3306
jdbc_url = "jdbc:mysql://{0}:{1}/{2}?characterEncoding=UTF-8".format(jdbc_hostname, jdbc_port, jdbc_database)

trips_glod_df.write \
    .format("jdbc").option("url", f"{jdbc_url}") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "trips_glod_df") \
    .option("user", f"{username}").option("password", f"{password}").mode("append").save()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
data_dir = "s3://"+ bucket +"/tlc_tripsdata/glod/trips_json/"
trips_glod_df.coalesce(1).write.format("json").mode('overwrite').save(f"{data_dir}")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…